In [16]:
from datetime import datetime

import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from torch.utils.data import Dataset

In [5]:
# GPUが使えるか確認
print(torch.cuda.is_available())

if torch.cuda.is_available():
    device = torch.device("cuda")
    # GPUの名前を確認
    print(torch.cuda.get_device_name(0))

    # GPUの数を確認
    print(torch.cuda.device_count())

False


In [2]:
# movieLensデータセットのダウンロード
# !wget -nc https://files.grouplens.org/datasets/movielens/ml-10m.zip -P data
# !unzip -n data/ml-10m.zip -d data/

In [39]:
# 映画の情報の読み込み
# m_cols = ["movie_id", "title", "genre"]
# movies = pd.read_csv(
#     "./data/ml-10M100K/movies.dat",
#     names=m_cols,
#     sep="::",
#     encoding="latin-1",
#     engine="python",
# )
# movies.to_csv("./data/movies.csv", index=False)

movies = pd.read_csv("./data/movies.csv")
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [40]:
# 評価値データの読み込み
# r_cols = ["user_id", "movie_id", "rating", "timestamp"]
# ratings = pd.read_csv(
#     "./data/ml-10M100K/ratings.dat",
#     names=r_cols,
#     sep="::",
#     encoding="latin-1",
#     engine="python",
# )

# valid_user_ids = sorted(ratings.user_id.unique())[:1000]
# ratings = ratings[ratings["user_id"].isin(valid_user_ids)]
# ratings.to_csv("./data/ratings.csv", index=False)

ratings = pd.read_csv("./data/ratings.csv")
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,122,5.0,838985046
1,1,185,5.0,838983525
2,1,231,5.0,838983392
3,1,292,5.0,838983421
4,1,316,5.0,838983392


In [41]:
def _encode_genres(genres_series):
    """
    ジャンルをワンホットエンコーディングする。
    Args:
        genres_series (pd.Series): ジャンルのカラム（例: "Adventure|Comedy"）
    Returns:
        pd.DataFrame: ジャンルのワンホットエンコードされたDataFrame
    """
    genres = set()
    for genres_str in genres_series:
        genres.update(genres_str.split("|"))
    genres = sorted(genres)

    # ジャンルごとにワンホットエンコード
    genre_df = pd.DataFrame(0, index=genres_series.index, columns=genres)
    for idx, genres_str in enumerate(genres_series):
        for genre in genres_str.split("|"):
            genre_df.at[idx, genre] = 1
    genre_df["movie_id_enc"] = genres_series.index
    return genre_df

In [42]:
def _create_user_sequences(ratings, max_seq_length):
    """
    ユーザーごとのインタラクションシーケンスを作成する。
    Args:
        ratings (pd.DataFrame): ユーザーインタラクションデータ
        max_seq_length (int): シーケンスの最大長
    Returns:
        list: 各ユーザーのシーケンスデータのリスト
    """
    user_sequences = []
    grouped = ratings.sort_values(["user_id_enc", "timestamp"]).groupby("user_id_enc")
    for user_id, group in grouped:
        item_ids = group["movie_id_enc"].tolist()
        timestamps = group["timestamp"].tolist()
        if len(item_ids) < 2:
            continue  # インタラクションが1つだけのユーザーはスキップ
        for i in range(1, len(item_ids)):
            start = max(0, i - max_seq_length)
            seq = item_ids[start:i]
            seq_timestamps = timestamps[start:i]
            target = item_ids[i]
            target_timestamp = timestamps[i]
            user_sequences.append(
                {
                    "user_id": user_id,
                    "sequence": seq,
                    "sequence_timestamps": seq_timestamps,
                    "target": target,
                    "target_timestamp": target_timestamp,
                }
            )
    return user_sequences

In [43]:
ratings_path = "./data/ratings.csv"
movies_path = "./data/movies.csv"
max_seq_length = 30
test_size = 0.2

# データの読み込み
ratings = pd.read_csv(ratings_path)  # user_id, movie_id, rating, timestamp
movies = pd.read_csv(movies_path)  # movie_id, title, genres

# 映画IDのエンコーディング
movie_encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)
ratings["movie_id_enc"] = movie_encoder.fit_transform(ratings[["movie_id"]])
movies["movie_id_enc"] = movie_encoder.transform(movies[["movie_id"]])

# ジャンルのワンホットエンコーディング
genre_encoder = _encode_genres(movies["genre"])
movies = movies.merge(genre_encoder, on="movie_id_enc")

# ユーザーIDのエンコーディング
user_encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)
ratings["user_id_enc"] = user_encoder.fit_transform(ratings[["user_id"]])

# # タイムスタンプの変換（例としてUNIXタイムスタンプをdatetimeに変換）
# ratings["timestamp"] = pd.to_datetime(ratings["timestamp"], unit="s")

# ユーザーごとのインタラクションシーケンスを作成
user_sequences = _create_user_sequences(ratings, max_seq_length)

# データの分割（トレーニングとテスト）
train_sequences, test_sequences = train_test_split(
    user_sequences, test_size=test_size, shuffle=False
)

# 現在はトレーニングデータを使用
data = train_sequences

In [56]:
sample = data[0]

user_id = sample["user_id"]
sequence = sample["sequence"]
sequence_timestamps = sample["sequence_timestamps"]
target = sample["target"]
target_timestamp = sample["target_timestamp"]

# シーケンスのパディング
seq_len = len(sequence)
if seq_len < 50:
    padding_length = 50 - seq_len
    sequence = [0] * padding_length + sequence  # 0でパディング
    sequence_timestamps = [0] * padding_length + sequence_timestamps
else:
    sequence = sequence[-50:]
    sequence_timestamps = sequence_timestamps[-50:]

# タイムスタンプを数値特徴量に変換（例: UNIXタイムスタンプ）
sequence_time_features = sequence_timestamps
target_time_feature = target_timestamp

# データをテンソルに変換
sequence_tensor = torch.tensor(sequence, dtype=torch.long)
target_tensor = torch.tensor(target, dtype=torch.long)
sequence_time_tensor = torch.tensor(sequence_time_features, dtype=torch.float)
target_time_tensor = torch.tensor(target_time_feature, dtype=torch.float)

In [63]:
class CARCADataset(Dataset):
    def __init__(self, ratings_path, movies_path, max_seq_length=50, test_size=0.2):
        """
        Args:
            ratings_path (str): ユーザーインタラクションデータのCSVファイルパス（例: "ratings.csv"）
            movies_path (str): 映画メタデータのCSVファイルパス（例: "movies.csv"）
            max_seq_length (int): ユーザーのインタラクションシーケンスの最大長
            test_size (float): データセットのテストサイズ割合
        """
        super(CARCADataset, self).__init__()

        # データの読み込み
        self.ratings = pd.read_csv(ratings_path)  # user_id, movie_id, rating, timestamp
        self.movies = pd.read_csv(movies_path)  # movie_id, title, genres

        # 映画IDのエンコーディング
        self.movie_encoder = OrdinalEncoder(
            handle_unknown="use_encoded_value", unknown_value=-1
        )
        self.ratings["movie_id_enc"] = self.movie_encoder.fit_transform(
            self.ratings[["movie_id"]]
        )
        self.movies["movie_id_enc"] = self.movie_encoder.transform(
            self.movies[["movie_id"]]
        )

        # ジャンルのワンホットエンコーディング
        self.genre_encoder = self._encode_genres(self.movies["genre"])
        self.movies = self.movies.merge(self.genre_encoder, on="movie_id_enc")

        # ユーザーIDのエンコーディング
        self.user_encoder = OrdinalEncoder(
            handle_unknown="use_encoded_value", unknown_value=-1
        )
        self.ratings["user_id_enc"] = self.user_encoder.fit_transform(
            self.ratings[["user_id"]]
        )

        # タイムスタンプの変換（例としてUNIXタイムスタンプをdatetimeに変換）
        # self.ratings["timestamp"] = pd.to_datetime(self.ratings["timestamp"], unit="s")

        # ユーザーごとのインタラクションシーケンスを作成
        self.user_sequences = self._create_user_sequences(self.ratings, max_seq_length)

        self.data = self.user_sequences

    def _encode_genres(self, genres_series):
        """
        ジャンルをワンホットエンコーディングする。
        Args:
            genres_series (pd.Series): ジャンルのカラム（例: "Adventure|Comedy"）
        Returns:
            pd.DataFrame: ジャンルのワンホットエンコードされたDataFrame
        """
        genres = set()
        for genres_str in genres_series:
            genres.update(genres_str.split("|"))
        genres = sorted(genres)

        # ジャンルごとにワンホットエンコード
        genre_df = pd.DataFrame(0, index=genres_series.index, columns=genres)
        for idx, genres_str in enumerate(genres_series):
            for genre in genres_str.split("|"):
                genre_df.at[idx, genre] = 1
        genre_df["movie_id_enc"] = genres_series.index
        return genre_df

    def _create_user_sequences(self, ratings, max_seq_length):
        """
        ユーザーごとのインタラクションシーケンスを作成する。
        Args:
            ratings (pd.DataFrame): ユーザーインタラクションデータ
            max_seq_length (int): シーケンスの最大長
        Returns:
            list: 各ユーザーのシーケンスデータのリスト
        """
        user_sequences = []
        grouped = ratings.sort_values(["user_id_enc", "timestamp"]).groupby(
            "user_id_enc"
        )
        for user_id, group in grouped:
            item_ids = group["movie_id_enc"].tolist()
            timestamps = group["timestamp"].tolist()
            if len(item_ids) < 2:
                continue  # インタラクションが1つだけのユーザーはスキップ
            for i in range(1, len(item_ids)):
                start = max(0, i - max_seq_length)
                seq = item_ids[start:i]
                seq_timestamps = timestamps[start:i]
                target = item_ids[i]
                target_timestamp = timestamps[i]
                user_sequences.append(
                    {
                        "user_id": user_id,
                        "sequence": seq,
                        "sequence_timestamps": seq_timestamps,
                        "target": target,
                        "target_timestamp": target_timestamp,
                    }
                )
        return user_sequences

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        """
        指定されたインデックスのデータサンプルを返す。
        Returns:
            dict: ユーザーID、シーケンスアイテムID、シーケンスアイテム属性、シーケンスタイムスタンプ、ターゲットアイテムID、ターゲットタイムスタンプ
        """
        sample = self.data[idx]

        user_id = sample["user_id"]
        sequence = sample["sequence"]
        sequence_timestamps = sample["sequence_timestamps"]
        target = sample["target"]
        target_timestamp = sample["target_timestamp"]

        # シーケンスのパディング
        seq_len = len(sequence)
        if seq_len < 50:
            padding_length = 50 - seq_len
            sequence = [0] * padding_length + sequence  # 0でパディング
            sequence_timestamps = [0] * padding_length + sequence_timestamps
        else:
            sequence = sequence[-50:]
            sequence_timestamps = sequence_timestamps[-50:]

        # タイムスタンプを数値特徴量に変換（例: UNIXタイムスタンプ）
        sequence_time_features = sequence_timestamps
        target_time_feature = target_timestamp

        # データをテンソルに変換
        sequence_tensor = torch.tensor(sequence, dtype=torch.long)
        target_tensor = torch.tensor(target, dtype=torch.long)
        sequence_time_tensor = torch.tensor(sequence_time_features, dtype=torch.int)
        target_time_tensor = torch.tensor(target_time_feature, dtype=torch.int)

        return {
            "user_id": torch.tensor(user_id, dtype=torch.long),
            "sequence": sequence_tensor,
            "sequence_time": sequence_time_tensor,
            "target": target_tensor,
            "target_time": target_time_tensor,
        }

In [64]:
dataset = CARCADataset(
    ratings_path=ratings_path, movies_path=movies_path, max_seq_length=50, test_size=0.2
)

# データサンプルの確認
sample = dataset[0]
print("ユーザーID:", sample["user_id"])
print("シーケンス:", sample["sequence"])
print("シーケンスタイムスタンプ:", sample["sequence_time"])
print("ターゲットアイテムID:", sample["target"])
print("ターゲットタイムスタンプ:", sample["target_time"])

# DataLoaderを使用する場合
from torch.utils.data import DataLoader

dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

for batch in dataloader:
    user_ids = batch["user_id"]
    sequences = batch["sequence"]
    sequences_time = batch["sequence_time"]
    targets = batch["target"]
    targets_time = batch["target_time"]
    # モデルへの入力として使用
    break  # 最初のバッチのみ確認

ユーザーID: tensor(0)
シーケンス: tensor([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0, 544])
シーケンスタイムスタンプ: tensor([        0,         0,         0,         0,         0,         0,
                0,         0,         0,         0,         0,         0,
                0,         0,         0,         0,         0,         0,
                0,         0,         0,         0,         0,         0,
                0,         0,         0,         0,         0,         0,
                0,         0,         0,         0,         0,         0,
                0,         0,         0,         0,         0,         0,
                0,         0,         0,         0,         0,         0,
                0, 838983339], dtype=torch.int32)
ターゲットアイテムID: tensor(215)
ターゲットタイムスタン